In [1]:
import pandas as pd
import numpy as np
import sys
import datetime as dt
import urllib
from sqlalchemy import create_engine

In [2]:
# https://www.medicalmarijuana.ohio.gov/DispensaryLocations

In [3]:
today_date = dt.date(2021, 10, 5)

today_year = today_date.strftime('%Y')
today_month = today_date.strftime('%m')
today_day = today_date.strftime('%d')

In [4]:
medical_disp_df = pd.read_excel(f'licenses/medical_licenses_{today_year}_{today_month}_{today_day}.xlsx')

In [5]:
# This creates the connection to the Ohio Cannabis files in SQL

params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=oh_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
engine_con = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))

### Rename Columns

In [6]:
medical_df_cleaned = medical_disp_df.rename(columns={
    'LicenseNumber':'license_number',
    'LicenseType':'license_description',
    'LicenseStatus':'license_status',
    'BusinessName':'name_legal',
    'LocationStreetAddress':'contact_address',
    'LocationCity':'contact_city',
    'LocationState':'contact_state',
    'LocationZip':'contact_zip',
    'LocationCounty':'contact_county',
    'Phone':'contact_phone',
    'Fax':'contact_fax',
    'FirstIssuanceDate':'date_issued',
    'LicenseEffectiveDate': 'date_effective',
    'LicenseExpirationDate':'date_expiration',
    'Designated Representative':'contact_representative',
    'Designated Representative License Number':'contact_representative_license_number'
})

In [7]:
if len(medical_df_cleaned['LicenseSubStatus'].unique()) > 1:
    print(error)

In [8]:
# This adds the roll_up data from SQL

sql_oh_roll = pd.read_sql('oh_roll', con=engine_con)
medical_df_cleaned = medical_df_cleaned.merge(sql_oh_roll, on='license_number', how='outer').fillna('')

In [9]:
# This is just here to clean the non numeric data out of contact_fax
# If the next cell won't run, add info here to fix that

medical_df_cleaned['contact_fax'] = medical_df_cleaned['contact_fax'].str.replace('BloomMedicinals', '')

In [10]:
medical_df_cleaned

,license_number,license_description,license_status,LicenseSubStatus,name_legal,contact_address,contact_city,contact_state,contact_zip,contact_county,contact_phone,contact_fax,date_issued,date_effective,date_expiration,BoardAction,contact_representative_license_number,contact_representative,roll_up_id,company_roll_up
0,MMD.0700030,Medical Marijuana Dispensary - Certificate of ...,Active,,Bloom Medicinals,1238 Conant St,Maumee,OH,43537-1608,Lucas,(561) 314-3949,,2019.09.04,2021.07.02,2023.07.01,No,MME.05200316,TAEYA SHERMAN,OH-00001,Bloom Medicinals
1,MMD.0700045,Medical Marijuana Dispensary - Certificate of ...,Active,,Bloom Medicinals,737 E North St,Akron,OH,44305-1115,Summit,(561) 314-3949,,2019.10.29,2021.07.02,2023.07.01,No,MME.05200625,Tena Johnson,OH-00001,Bloom Medicinals
2,MMD.0700031,Medical Marijuana Dispensary - Certificate of ...,Active,,Bloom Medicinals,403 S Main St,Seven Mile,OH,45011,Butler,(561) 314-3949,Bloom Medicinals,2019.09.10,2021.07.02,2023.07.01,No,MME.05200302,Andre' McMillen,OH-00001,Bloom Medicinals
3,MMD.0700018,Medical Marijuana Dispensary - Certificate of ...,Active,,Bloom Medicinals,382 Blackbrook Rd,Painesville,OH,44077-1294,Lake,(561) 314-3949,,2019.05.16,2021.07.02,2023.07.01,No,MME.05200525,Christopher Chagnon,OH-00001,Bloom Medicinals
4,MMD.0700025,Medical Marijuana Dispensary - Certificate of ...,Active,,Bloom Medicinals,1361 Georgesville Rd,Columbus,OH,43228-3611,Franklin,(561) 314-3949,Bloom Medicinals,2019.07.26,2021.07.02,2023.07.01,No,MME.05200353,Allison Chercourt,OH-00001,Bloom Medicinals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,MMD.0700058,Medical Marijuana Dispensary - Certificate of ...,Active,,Zen Leaf Bowling Green,1155 N Main St,Bowling Green,OH,43402-1310,Wood,(000) 000-0000,,2020.12.29,2020.12.29,2023.07.01,No,MME.05200550,Tonya Arce-Kulakowski,OH-00024,Zen Leaf
59,MMD.0700068,Medical Marijuana Dispensary - Certificate of ...,Active,,Zen Leaf Canton,1851 Steinway Blvd SE,Canton,OH,44707-2643,Stark,(224) 622-8770,,2021.06.30,2021.06.30,2023.07.01,No,MME.05200248,Darion Cole,OH-00024,Zen Leaf
60,MMD.0700066,Medical Marijuana Dispensary - Certificate of ...,Active,,Zen Leaf Cincinnati,8420 Vine St,Cincinnati,OH,45216-1116,Hamilton,(224) 622-8770,,2021.07.01,2021.07.01,2023.07.01,No,MME.05200402,Andre Foster Lopez,OH-00024,Zen Leaf
61,MMD.0700069,Medical Marijuana Dispensary - Certificate of ...,Active,,Zen Leaf Dayton,5604 Airway Rd,Riverside,OH,45431-1506,Montgomery,(224) 622-8770,,2021.07.08,2021.07.08,2023.07.01,No,MME.05200089,SARA HAWKINS,OH-00024,Zen Leaf


In [11]:
medical_df_cleaned['contact_phone'] = medical_df_cleaned['contact_phone'].str.replace('(', '').str.replace(')', '').str.replace('-', '').str.replace(' ', '')
medical_df_cleaned['contact_fax'] = medical_df_cleaned['contact_fax'].str.replace('(', '').str.replace(')', '').str.replace('-', '').str.replace(' ', '').fillna('')

In [12]:
medical_df_cleaned = medical_df_cleaned[['license_number', 'license_description', 'license_status',
                    'LicenseSubStatus', 'name_legal', 'roll_up_id', 'company_roll_up', 
                    'contact_address', 'contact_city', 'contact_state', 
                    'contact_zip', 'contact_county', 'contact_phone', 'contact_fax', 
                    'date_issued', 'date_effective', 'date_expiration', 'BoardAction', 
                    'contact_representative_license_number', 'contact_representative']]

In [13]:
medical_df_cleaned['date_expiration'] = pd.to_datetime(medical_df_cleaned['date_expiration'])
medical_df_cleaned['date_effective'] = pd.to_datetime(medical_df_cleaned['date_effective'])
medical_df_cleaned['date_issued'] = pd.to_datetime(medical_df_cleaned['date_issued'])

In [14]:
medical_df_cleaned

,license_number,license_description,license_status,LicenseSubStatus,name_legal,roll_up_id,company_roll_up,contact_address,contact_city,contact_state,contact_zip,contact_county,contact_phone,contact_fax,date_issued,date_effective,date_expiration,BoardAction,contact_representative_license_number,contact_representative
0,MMD.0700030,Medical Marijuana Dispensary - Certificate of ...,Active,,Bloom Medicinals,OH-00001,Bloom Medicinals,1238 Conant St,Maumee,OH,43537-1608,Lucas,5613143949,,2019-09-04,2021-07-02,2023-07-01,No,MME.05200316,TAEYA SHERMAN
1,MMD.0700045,Medical Marijuana Dispensary - Certificate of ...,Active,,Bloom Medicinals,OH-00001,Bloom Medicinals,737 E North St,Akron,OH,44305-1115,Summit,5613143949,,2019-10-29,2021-07-02,2023-07-01,No,MME.05200625,Tena Johnson
2,MMD.0700031,Medical Marijuana Dispensary - Certificate of ...,Active,,Bloom Medicinals,OH-00001,Bloom Medicinals,403 S Main St,Seven Mile,OH,45011,Butler,5613143949,BloomMedicinals,2019-09-10,2021-07-02,2023-07-01,No,MME.05200302,Andre' McMillen
3,MMD.0700018,Medical Marijuana Dispensary - Certificate of ...,Active,,Bloom Medicinals,OH-00001,Bloom Medicinals,382 Blackbrook Rd,Painesville,OH,44077-1294,Lake,5613143949,,2019-05-16,2021-07-02,2023-07-01,No,MME.05200525,Christopher Chagnon
4,MMD.0700025,Medical Marijuana Dispensary - Certificate of ...,Active,,Bloom Medicinals,OH-00001,Bloom Medicinals,1361 Georgesville Rd,Columbus,OH,43228-3611,Franklin,5613143949,BloomMedicinals,2019-07-26,2021-07-02,2023-07-01,No,MME.05200353,Allison Chercourt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,MMD.0700058,Medical Marijuana Dispensary - Certificate of ...,Active,,Zen Leaf Bowling Green,OH-00024,Zen Leaf,1155 N Main St,Bowling Green,OH,43402-1310,Wood,0000000000,,2020-12-29,2020-12-29,2023-07-01,No,MME.05200550,Tonya Arce-Kulakowski
59,MMD.0700068,Medical Marijuana Dispensary - Certificate of ...,Active,,Zen Leaf Canton,OH-00024,Zen Leaf,1851 Steinway Blvd SE,Canton,OH,44707-2643,Stark,2246228770,,2021-06-30,2021-06-30,2023-07-01,No,MME.05200248,Darion Cole
60,MMD.0700066,Medical Marijuana Dispensary - Certificate of ...,Active,,Zen Leaf Cincinnati,OH-00024,Zen Leaf,8420 Vine St,Cincinnati,OH,45216-1116,Hamilton,2246228770,,2021-07-01,2021-07-01,2023-07-01,No,MME.05200402,Andre Foster Lopez
61,MMD.0700069,Medical Marijuana Dispensary - Certificate of ...,Active,,Zen Leaf Dayton,OH-00024,Zen Leaf,5604 Airway Rd,Riverside,OH,45431-1506,Montgomery,2246228770,,2021-07-08,2021-07-08,2023-07-01,No,MME.05200089,SARA HAWKINS


### Send to CSV

In [15]:
# medical_df_cleaned.to_csv(f'edited_files/df_{today_year}_{today_month}_{today_day}_to_edit.csv', index=False)

##### Open CSV and Add company_roll_ups

In [16]:
if sum(pd.read_csv(f'edited_files/df_{today_year}_{today_month}_{today_day}_to_edit.csv')['company_roll_up'].isnull()) == 0:
    df_with_comp_roll = pd.read_csv(f'edited_files/df_{today_year}_{today_month}_{today_day}_to_edit.csv')
else:
    print(error)

In [17]:
df_with_comp_roll = df_with_comp_roll.merge(medical_df_cleaned[['license_number', 'date_effective']])

### Add roll_up_id

In [18]:
df_with_comp_roll['roll_up_id'] = df_with_comp_roll['roll_up_id'].fillna('')

In [19]:
df_empty_roll = df_with_comp_roll[df_with_comp_roll['roll_up_id'] == '']
df_not_empty_roll = df_with_comp_roll[df_with_comp_roll['roll_up_id'] != '']

In [20]:
current_rolls = []
for roll in df_not_empty_roll['roll_up_id']:
    current_rolls.append(int(roll[3:]))

In [21]:
max_roll = max(current_rolls)
max_roll = 0

In [22]:
comp_roll_list = list(df_empty_roll['company_roll_up'].unique())

In [23]:
roll_id_list = []
for comp in comp_roll_list:
    max_roll+=1
    roll_id_list.append(max_roll)

In [24]:
roll_id_list

[1]

In [25]:
oh_roll_id_list = []

for roll in roll_id_list:
    while len(str(roll)) < 5:
        roll = '0' + str(roll)
    roll = 'OH-' + str(roll)
    oh_roll_id_list.append(roll)

In [26]:
oh_roll_id_list

['OH-00001']

In [27]:
df_empty_roll.reset_index().drop(columns='index')

,license_number,license_description,license_status,LicenseSubStatus,name_legal,roll_up_id,company_roll_up,contact_address,contact_city,contact_state,...,contact_county,contact_phone,contact_fax,date_issued,LicenseEffectiveDate,date_expiration,BoardAction,contact_representative_license_number,contact_representative,date_effective
0,MMD.0700080,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,"High Street Capital Partners, LLC",,"High Street Capital Partners, LLC",46 S Summit St,Akron,OH,...,Summit,3303682144,NaN,2021.10.01,2021.10.01,2023.07.01,No,MME.05200454,CASSANDRA CHAPMAN,2021-10-01
1,MMD.0700083,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,"High Street Capital Partners, LLC",,"High Street Capital Partners, LLC",30133 Euclid Ave,Wickliffe,OH,...,Lake,4406710420,NaN,2021.10.01,2021.10.01,2023.07.01,No,MME.05200290,EMILY MUHIC,2021-10-01
2,MMD.0700082,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,"High Street Capital Partners, LLC",,"High Street Capital Partners, LLC",3865 Lakeside Ave E,Cleveland,OH,...,Cuyahoga,2169306550,NaN,2021.10.01,2021.10.01,2023.07.01,No,MME.05200479,GYEN MUSGRAVE,2021-10-01
3,MMD.0700081,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,"High Street Capital Partners, LLC",,"High Street Capital Partners, LLC",115 Vine St,Columbus,OH,...,Franklin,6149242155,NaN,2021.10.01,2021.10.01,2023.07.01,No,MME.05200829,STEVEN MEEK,2021-10-01
4,MMD.0700084,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,"High Street Capital Partners, LLC",,"High Street Capital Partners, LLC",3840 Greentree Ave SW,Canton,OH,...,Stark,3306237430,NaN,2021.10.01,2021.10.01,2023.07.01,No,MME.05200034,KALEE ECKLES,2021-10-01


In [28]:
for n in range(len(df_empty_roll)):
    df_empty_roll['roll_up_id'].iloc[n] = oh_roll_id_list[comp_roll_list.index(df_empty_roll['company_roll_up'].iloc[n])]

C:\Users\nolan\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\nolan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
df_empty_roll

,license_number,license_description,license_status,LicenseSubStatus,name_legal,roll_up_id,company_roll_up,contact_address,contact_city,contact_state,...,contact_county,contact_phone,contact_fax,date_issued,LicenseEffectiveDate,date_expiration,BoardAction,contact_representative_license_number,contact_representative,date_effective
29,MMD.0700080,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,"High Street Capital Partners, LLC",OH-00001,"High Street Capital Partners, LLC",46 S Summit St,Akron,OH,...,Summit,3303682144,NaN,2021.10.01,2021.10.01,2023.07.01,No,MME.05200454,CASSANDRA CHAPMAN,2021-10-01
30,MMD.0700083,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,"High Street Capital Partners, LLC",OH-00001,"High Street Capital Partners, LLC",30133 Euclid Ave,Wickliffe,OH,...,Lake,4406710420,NaN,2021.10.01,2021.10.01,2023.07.01,No,MME.05200290,EMILY MUHIC,2021-10-01
31,MMD.0700082,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,"High Street Capital Partners, LLC",OH-00001,"High Street Capital Partners, LLC",3865 Lakeside Ave E,Cleveland,OH,...,Cuyahoga,2169306550,NaN,2021.10.01,2021.10.01,2023.07.01,No,MME.05200479,GYEN MUSGRAVE,2021-10-01
32,MMD.0700081,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,"High Street Capital Partners, LLC",OH-00001,"High Street Capital Partners, LLC",115 Vine St,Columbus,OH,...,Franklin,6149242155,NaN,2021.10.01,2021.10.01,2023.07.01,No,MME.05200829,STEVEN MEEK,2021-10-01
33,MMD.0700084,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,"High Street Capital Partners, LLC",OH-00001,"High Street Capital Partners, LLC",3840 Greentree Ave SW,Canton,OH,...,Stark,3306237430,NaN,2021.10.01,2021.10.01,2023.07.01,No,MME.05200034,KALEE ECKLES,2021-10-01


In [30]:
df_empty_roll['date_uploaded'] = today_date
df_empty_roll['is_current'] = 1

C:\Users\nolan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\nolan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
df_roll_combined = df_empty_roll.append(df_not_empty_roll).sort_values(by='name_legal').reset_index().drop(columns='index')

In [32]:
df_roll_combined['date_uploaded'] = today_date
df_roll_combined['is_current'] = 1

In [33]:
df_roll_combined

,license_number,license_description,license_status,LicenseSubStatus,name_legal,roll_up_id,company_roll_up,contact_address,contact_city,contact_state,...,contact_fax,date_issued,LicenseEffectiveDate,date_expiration,BoardAction,contact_representative_license_number,contact_representative,date_effective,date_uploaded,is_current
0,MMD.0700030,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,Bloom Medicinals,OH-00001,Bloom Medicinals,1238 Conant St,Maumee,OH,...,NaN,2019.09.04,2021.07.02,2023.07.01,No,MME.05200316,TAEYA SHERMAN,2021-07-02,2021-10-05,1
1,MMD.0700045,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,Bloom Medicinals,OH-00001,Bloom Medicinals,737 E North St,Akron,OH,...,NaN,2019.10.29,2021.07.02,2023.07.01,No,MME.05200625,Tena Johnson,2021-07-02,2021-10-05,1
2,MMD.0700031,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,Bloom Medicinals,OH-00001,Bloom Medicinals,403 S Main St,Seven Mile,OH,...,BloomMedicinals,2019.09.10,2021.07.02,2023.07.01,No,MME.05200302,Andre' McMillen,2021-07-02,2021-10-05,1
3,MMD.0700018,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,Bloom Medicinals,OH-00001,Bloom Medicinals,382 Blackbrook Rd,Painesville,OH,...,NaN,2019.05.16,2021.07.02,2023.07.01,No,MME.05200525,Christopher Chagnon,2021-07-02,2021-10-05,1
4,MMD.0700025,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,Bloom Medicinals,OH-00001,Bloom Medicinals,1361 Georgesville Rd,Columbus,OH,...,BloomMedicinals,2019.07.26,2021.07.02,2023.07.01,No,MME.05200353,Allison Chercourt,2021-07-02,2021-10-05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,MMD.0700058,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,Zen Leaf Bowling Green,OH-00024,Zen Leaf,1155 N Main St,Bowling Green,OH,...,NaN,2020.12.29,2020.12.29,2023.07.01,No,MME.05200550,Tonya Arce-Kulakowski,2020-12-29,2021-10-05,1
59,MMD.0700068,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,Zen Leaf Canton,OH-00024,Zen Leaf,1851 Steinway Blvd SE,Canton,OH,...,NaN,2021.06.30,2021.06.30,2023.07.01,No,MME.05200248,Darion Cole,2021-06-30,2021-10-05,1
60,MMD.0700066,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,Zen Leaf Cincinnati,OH-00024,Zen Leaf,8420 Vine St,Cincinnati,OH,...,NaN,2021.07.01,2021.07.01,2023.07.01,No,MME.05200402,Andre Foster Lopez,2021-07-01,2021-10-05,1
61,MMD.0700069,Medical Marijuana Dispensary - Certificate of ...,Active,NaN,Zen Leaf Dayton,OH-00024,Zen Leaf,5604 Airway Rd,Riverside,OH,...,NaN,2021.07.08,2021.07.08,2023.07.01,No,MME.05200089,SARA HAWKINS,2021-07-08,2021-10-05,1


### Remove Old SQL

In [34]:
df_roll_combined = df_roll_combined[~df_roll_combined['license_number'].isin(pd.read_sql('oh_main', con=engine_con)['license_number'].unique())]

In [35]:
df_roll_combined['date_issued'] = pd.to_datetime(df_roll_combined['date_issued'])
df_roll_combined['date_effective'] = pd.to_datetime(df_roll_combined['date_effective'])
df_roll_combined['date_expiration'] = pd.to_datetime(df_roll_combined['date_expiration'])

C:\Users\nolan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\nolan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\nolan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

### Seperate for SQL and Send

In [36]:
for_sql_main = df_roll_combined[['license_number', 'name_legal', 'license_description', 'license_status', 'date_issued', 'date_effective', 'date_expiration', 'date_uploaded', 'is_current']]
for_sql_roll = df_roll_combined[['license_number', 'roll_up_id', 'company_roll_up']]
for_sql_contact = df_roll_combined[['license_number', 'contact_address', 'contact_city', 'contact_state', 'contact_zip', 'contact_county', 'contact_phone', 'contact_fax', 'contact_representative', 'contact_representative_license_number']]

In [37]:
for_sql_main

,license_number,name_legal,license_description,license_status,date_issued,date_effective,date_expiration,date_uploaded,is_current
29,MMD.0700080,"High Street Capital Partners, LLC",Medical Marijuana Dispensary - Certificate of ...,Active,2021-10-01,2021-10-01,2023-07-01,2021-10-05,1
30,MMD.0700081,"High Street Capital Partners, LLC",Medical Marijuana Dispensary - Certificate of ...,Active,2021-10-01,2021-10-01,2023-07-01,2021-10-05,1
31,MMD.0700084,"High Street Capital Partners, LLC",Medical Marijuana Dispensary - Certificate of ...,Active,2021-10-01,2021-10-01,2023-07-01,2021-10-05,1
32,MMD.0700082,"High Street Capital Partners, LLC",Medical Marijuana Dispensary - Certificate of ...,Active,2021-10-01,2021-10-01,2023-07-01,2021-10-05,1
33,MMD.0700083,"High Street Capital Partners, LLC",Medical Marijuana Dispensary - Certificate of ...,Active,2021-10-01,2021-10-01,2023-07-01,2021-10-05,1
38,MMD.0700075,Ohio Provisions,Medical Marijuana Dispensary - Certificate of ...,Active,2021-10-01,2021-10-01,2023-07-01,2021-10-05,1


In [38]:
print(stop)

NameError: name 'stop' is not defined

In [39]:
for_sql_main.to_sql('oh_main', con=engine_con, index=False, if_exists='append')
for_sql_roll.to_sql('oh_roll', con=engine_con, index=False, if_exists='append')
for_sql_contact.to_sql('oh_contact', con=engine_con, index=False, if_exists='append')